## WFP-01-03-02 CHIRPS Rainfall Estimates (RFE) - Aggregations

 This application generates Rainfall Estimates (RFE) aggregations, from CHIRPS RFE 5km resolution, compared to a reference period.

### <a name="objective">Objective 

The objective of this code is to determine:
    - Sum of daily data over the past N days, derived every 10 days (N = 10, 30, 60, 90, 120, 150, 180, 270, 365 days)
    - Counts of daily data above 1mm over the past N days, derived every 10 days (N = 30, 60, 90 days).
    - Longest sequence of daily values < 2mm ("dry spell") within the last N days, derived every 10 days (N = 30, 60, 90 days).


### <a name="service">Service definition

In [1]:
service = dict([('title', 'CHIRPS Rainfall Estimates (RFE) - Aggregations'),
                ('abstract', 'TBD'),
                ('id', 'wfp-01-03-02')])

### <a name="runtime">Runtime parameter definition

**Input references**

This is the CHIRPS stack catalogue references

In [2]:
input_references = 'https://catalog.terradue.com/chirps/search?format=atom&uid=chirps-v2.0.2017.03.10'

In [3]:
N_10 = dict([('id', 'N_10'),
                          ('value', 'True'),
                          ('title', '10 Day Aggregation'),
                          ('abstract', 'Get a 10 day aggregation')])

In [4]:
N_30 = dict([('id', 'N_30'),
                          ('value', 'True'),
                          ('title', '30 Day Aggregation'),
                          ('abstract', 'Get a 30 day aggregation')])

In [5]:
N_60 = dict([('id', 'N_60'),
                          ('value', 'False'),
                          ('title', '60 Day Aggregation'),
                          ('abstract', 'Get a 60 day aggregation')])

In [6]:
N_90 = dict([('id', 'N_90'),
                          ('value', 'False'),
                          ('title', '90 Day Aggregation'),
                          ('abstract', 'Get a 90 day aggregation')])

In [7]:
N_120 = dict([('id', 'N_120'),
                          ('value', 'False'),
                          ('title', '120 Day Aggregation'),
                          ('abstract', 'Get a 120 day aggregation')])

In [8]:
N_150 = dict([('id', 'N_150'),
                          ('value', 'False'),
                          ('title', '150 Day Aggregation'),
                          ('abstract', 'Get a 150 day aggregation')])

In [9]:
N_180 = dict([('id', 'N_180'),
                          ('value', 'False'),
                          ('title', '180 Day Aggregation'),
                          ('abstract', 'Get a 180 day aggregation')])

In [10]:
N_270 = dict([('id', 'N_270'),
                          ('value', 'False'),
                          ('title', '270 Day Aggregation'),
                          ('abstract', 'Get a 270 day aggregation')])

In [11]:
N_365 = dict([('id', 'N_365'),
                          ('value', 'False'),
                          ('title', '365 Day Aggregation'),
                          ('abstract', 'Get a 365 day aggregation')])

In [12]:
regionOfInterest = dict([('id', 'regionOfInterest'),
                          ('value', 'POLYGON((11.5030755518998 -11.1141633706909,41.0343255518998 -11.1141633706909,41.0343255518998 -34.9763656693858,11.5030755518998 -34.9763656693858,11.5030755518998 -11.1141633706909))'),
                          ('title', 'WKT Polygon for the Region of Interest'),
                          ('abstract', 'Set the value of WKT Polygon')])

In [13]:
nameOfRegion = dict([('id', 'nameOfRegion'),
                     ('value',  ''),
                     ('title', 'Name of Region'),
                     ('abstract', 'Name of the region of interest'),
                     ('minOccurs', '1')])

In [14]:
lta_url = dict([('id', 'lta_url'),
                ('value', 'https://catalog.terradue.com//better-wfp-00009/series/results/search'),
                ('title', 'Catalogue Url for the LTA products'),
                ('abstract', 'Catalogue Url for anomalies')])

### <a name="workflow">Workflow

#### Import the packages required for processing the data

In [15]:
from osgeo import gdal, ogr, osr
from geopandas import GeoDataFrame
from datetime import datetime
import gzip
import cioppy
import shutil
import sys
import numpy as np
import pandas as pd
import math
import re
import os
import requests
from shapely.wkt import loads
sys.path.append(os.getcwd())
sys.path.append('/application/notebook/libexec/')
from aux_functions import matrix_sum, mask_matrix, crop_image, write_output_image, get_matrix_list
print(sys.path)
ciop = cioppy.Cioppy()

['', '/opt/anaconda/lib/python27.zip', '/opt/anaconda/lib/python2.7', '/opt/anaconda/lib/python2.7/plat-linux2', '/opt/anaconda/lib/python2.7/lib-tk', '/opt/anaconda/lib/python2.7/lib-old', '/opt/anaconda/lib/python2.7/lib-dynload', '/opt/anaconda/lib/python2.7/site-packages/Glymur-0.8.6-py2.7.egg', '/opt/anaconda/lib/python2.7/site-packages/click_plugins-1.0.3-py2.7.egg', '/opt/anaconda/lib/python2.7/site-packages/setuptools-23.0.0-py2.7.egg', '/opt/anaconda/lib/python2.7/site-packages/vincent-0.4.4-py2.7.egg', '/opt/anaconda/lib/python2.7/site-packages', '/opt/anaconda/lib/python2.7/site-packages/PIL', '/opt/anaconda/lib/python2.7/site-packages/IPython/extensions', '/home/rirr/.ipython', '/workspace/wfp-01-03-02/src/main/app-resources/notebook/libexec', '/application/notebook/libexec/']


In [16]:
def get_info(row, search_params):
    search = ciop.search(end_point=row['catalogue_url'], 
                                  params=search_params,
                                  output_fields='self,identifier,startdate,enclosure',
                                  model='GeoTime')[0]
    
    series = pd.Series(search)
    
    series['startdate'] = pd.to_datetime(series['startdate'])
    
    return series

In [17]:
def get_lta_list(url, search_params):
    search = ciop.search(end_point=url,
                         params=search_params,
                         output_fields='self,identifier,enclosure,title,startdate,enddate,wkt,updated',
                         model='GeoTime')
    return search

In [18]:
def get_lta_info(row, search_params):
    search = ciop.search(end_point=row['catalogue_url'], 
                                  params=search_params,
                                  output_fields='self,identifier,startdate,enclosure,title,enddate,wkt',
                                  model='GeoTime')[0]
    
    series = pd.Series(search)
    
    series['startdate'] = pd.to_datetime(series['startdate'])
    series['enddate'] = pd.to_datetime(series['enddate'])
    
    return series

In [19]:
if isinstance(input_references, str):
    input_references = [input_references]

nameOfRegion = nameOfRegion['value']
#nameOfRegion = 'SouthernAfrica'



gpd_data = GeoDataFrame(input_references,
                       columns=['catalogue_url'])

region_of_interest = regionOfInterest['value']
gpd_data = gpd_data.sort_values(by='catalogue_url')
start_date = re.findall('\d{4}\.\d{2}\.\d{2}', gpd_data.iloc[0]['catalogue_url'])[0].replace('.', '-')
end_date = re.findall('\d{4}\.\d{2}\.\d{2}', gpd_data.iloc[-1]['catalogue_url'])[0].replace('.', '-')
print(start_date)
print(end_date)
print(len(input_references))
search_params =  dict([('start', start_date),
                      ('stop', end_date),
                      ('count', len(input_references))])
gpd_final = gpd_data.apply(lambda row: get_info(row, search_params), axis=1)
gpd_final.head()
#Insert code here to convert the LTA List to a Geopandas Data Frame
#lta_data = GeoDataFrame(LTA_list, columns=['catalogue_url'])

print(gpd_final.iloc[0]['enclosure'])

2015-01-01
2015-01-31
31
https://store.terradue.com/chirps/2015/01/01/files/v1/chirps-v2.0.2015.01.01.tif.gz


In [20]:
gpd_final.head()

,enclosure,identifier,self,startdate
30,https://store.terradue.com/chirps/2015/01/01/f...,chirps-v2.0.2015.01.01,https://catalog.terradue.com/chirps/search?for...,2015-01-01
29,https://store.terradue.com/chirps/2015/01/02/f...,chirps-v2.0.2015.01.02,https://catalog.terradue.com/chirps/search?for...,2015-01-02
28,https://store.terradue.com/chirps/2015/01/03/f...,chirps-v2.0.2015.01.03,https://catalog.terradue.com/chirps/search?for...,2015-01-03
27,https://store.terradue.com/chirps/2015/01/04/f...,chirps-v2.0.2015.01.04,https://catalog.terradue.com/chirps/search?for...,2015-01-04
26,https://store.terradue.com/chirps/2015/01/05/f...,chirps-v2.0.2015.01.05,https://catalog.terradue.com/chirps/search?for...,2015-01-05


In [21]:
lta_params = dict([('start', start_date + 'T00:00:00.0000000Z'),
                   ('stop', end_date + 'T00:00:00.0000000Z'),
                   ('geom', region_of_interest),
                   ('count', 150)])
print(lta_params)
lta_data = get_lta_list(lta_url['value'], lta_params)
#lta_data = GeoDataFrame(lta_list, columns=['catalogue_url'])
#lta_data = lta_data.apply(lambda row: get_lta_info(row,[]), axis=1)

lta_data = GeoDataFrame.from_dict(lta_data)
lta_data['startdate'] = pd.to_datetime(lta_data['startdate'])
lta_data['enddate'] = pd.to_datetime(lta_data['enddate'])
lta_data['wkt'] = lta_data['wkt'].apply(lambda row: loads(row))
lta_data.head(20)

{'count': 150, 'start': '2015-01-01T00:00:00.0000000Z', 'geom': 'POLYGON((11.5030755518998 -11.1141633706909,41.0343255518998 -11.1141633706909,41.0343255518998 -34.9763656693858,11.5030755518998 -34.9763656693858,11.5030755518998 -11.1141633706909))', 'stop': '2015-01-31T00:00:00.0000000Z'}


,enclosure,enddate,identifier,self,startdate,title,updated,wkt
0,https://store.terradue.com/better-wfp-00009/_r...,2017-01-31,0C12098705F6C606B0A946D85A7A39770D5D461F,https://catalog.terradue.com//better-wfp-00009...,2015-01-01,Output LTA_CHIRPSv2_SouthernAfrica_N30_dryspel...,2019-03-11T15:39:44.3101810+00:00,"POLYGON ((11.5030755518998 -11.1141633706909, ..."
1,https://store.terradue.com/better-wfp-00009/_r...,2017-01-31,147931514ADF3BD9A3EB3C72F097224304197E8C,https://catalog.terradue.com//better-wfp-00009...,2015-01-10,Output LTA_LAI_SouthernAfrica_N3_averages_1-10...,2019-03-11T16:09:09.0879070+00:00,"POLYGON ((11.5030755518998 -11.1141633706909, ..."
2,https://store.terradue.com/better-wfp-00009/_r...,2017-01-31,55813C2ADE9BD1824A03AF21CE7740A22A8CD0BF,https://catalog.terradue.com//better-wfp-00009...,2015-01-10,Output LTA_FAPAR_SouthernAfrica_N3_averages_1-...,2019-03-11T16:09:09.0879080+00:00,"POLYGON ((11.5030755518998 -11.1141633706909, ..."
3,https://store.terradue.com/better-wfp-00009/_r...,2017-01-31,7558809279D798348CD475AACAA52869D963A76E,https://catalog.terradue.com//better-wfp-00009...,2015-01-01,Output LTA_CHIRPSv2_SouthernAfrica_N30_daystot...,2019-03-11T15:39:44.3101810+00:00,"POLYGON ((11.5030755518998 -11.1141633706909, ..."
4,https://store.terradue.com/better-wfp-00009/_r...,2017-01-31,86A0BE56B5DFBDE03197DCE0E3491F36BDA769A2,https://catalog.terradue.com//better-wfp-00009...,2015-01-01,Output LTA_CHIRPSv2_SouthernAfrica_N30_countab...,2019-03-11T15:39:44.3101820+00:00,"POLYGON ((11.5030755518998 -11.1141633706909, ..."
5,https://store.terradue.com/better-wfp-00009/_r...,2017-01-31,9BF5187EBF0CE31AEB91766552F8AB76F7C0B690,https://catalog.terradue.com//better-wfp-00009...,2015-01-10,Output LTA_LAI_SouthernAfrica_N3_maxvalues_1-1...,2019-03-11T16:09:09.0879070+00:00,"POLYGON ((11.5030755518998 -11.1141633706909, ..."
6,https://store.terradue.com/better-wfp-00009/_r...,2017-01-31,F4DE56825DB1A70F233AE41C21F453280DB0B8E9,https://catalog.terradue.com//better-wfp-00009...,2015-01-10,Output LTA_FAPAR_SouthernAfrica_N3_maxvalues_1...,2019-03-11T16:09:09.0879080+00:00,"POLYGON ((11.5030755518998 -11.1141633706909, ..."
7,https://store.terradue.com/better-wfp-00009/_r...,2017-01-10,1036A9BE825E14EA3ECADB5B0298537ED59B9134,https://catalog.terradue.com//better-wfp-00009...,2015-01-10,Output LTA_LAI_SouthernAfrica_N3_maxvalues_1-1...,2019-03-08T16:19:09.2599870+00:00,"POLYGON ((11.5030755518998 -11.1141633706909, ..."
8,https://store.terradue.com/better-wfp-00009/_r...,2017-01-10,150A971BED4D23C6A2405E54B44B99FC3AF7A386,https://catalog.terradue.com//better-wfp-00009...,2015-01-10,Output LTA_FAPAR_SouthernAfrica_N3_averages_1-...,2019-03-08T16:19:09.2599880+00:00,"POLYGON ((11.5030755518998 -11.1141633706909, ..."
9,https://store.terradue.com/better-wfp-00009/_r...,2017-01-10,636448E3A888FA320A99809B1C101BE0ABB1FC2D,https://catalog.terradue.com//better-wfp-00009...,2015-01-10,Output LTA_LAI_SouthernAfrica_N3_averages_1-10...,2019-03-08T16:19:09.2599870+00:00,"POLYGON ((11.5030755518998 -11.1141633706909, ..."


In [22]:
def get_lta_from_dataframe(dataframe, region_name, product_type, aggregation, start_day, end_day, start_month, end_month, wkt):
    region_polygon = loads(wkt)
    dataframe_agr = dataframe[(dataframe['title'].str.contains(aggregation)) &
                              (dataframe['title'].str.contains(product_type)) &
                              (dataframe['title'].str.contains(region_name))]
    period_lta = dataframe_agr[(dataframe_agr['startdate'].dt.day == start_day) & 
                     (dataframe_agr['startdate'].dt.month == start_month) & 
                     (dataframe_agr['enddate'].dt.day == end_day) & 
                     (dataframe_agr['enddate'].dt.month == end_month) &
                     (dataframe_agr['wkt'] == region_polygon)]
    
    if len(period_lta.index.values) > 1:
        outdated_indexes = period_lta[period_lta['updated'] != max(period_lta['updated'])].index.values
        period_lta = period_lta.drop(outdated_indexes)
    return period_lta['enclosure'].tolist()


In [23]:
def calc_aggregations(product_list, N_value, region_of_interest):
    mask_no_data_value = 0
    sum_result = 0
    count_above_one = 0
    max_sequence = 0
    temp_mat = 0
    regions_below_two = 0
    projection = None
    geo_transform = None
    
    for chirp_product_url in product_list:
        # uncompressed data
        chirp_product = (chirp_product_url.split('/')[-1]).split('.gz')[0]
        print(chirp_product)
        cropped_product_path = 'crop_' + chirp_product
        try:
            crop_image(chirp_product_url, region_of_interest, cropped_product_path)
            # Read GeoTIFF as an array
            dataset = gdal.Open(cropped_product_path)
            product_array = dataset.GetRasterBand(1).ReadAsArray()
            no_data_value = dataset.GetRasterBand(1).GetNoDataValue()
            geo_transform = dataset.GetGeoTransform()
            projection = dataset.GetProjection()
            ## Create mask of no_data_values
            if isinstance(mask_no_data_value, int):
                mask_no_data_value = np.where(product_array == no_data_value, 1, 0)
            else:
                temp_mask = np.where(product_array == no_data_value, 1, 0)
                mask_no_data_value = matrix_sum(mask_no_data_value, temp_mask)

            ## Create iteratively the sum array
            sum_result = matrix_sum(sum_result, product_array, no_data_value)
            ## Create iteratively the array with the counts of daily data above 1mm
            if N_value == 30 or N_value == 60 or N_value == 90:
                regions_above_one = mask_matrix(product_array, 1, True, no_data_value)
                count_above_one = matrix_sum(count_above_one, regions_above_one)

                ## Create iteratively the array with the longest sequence of daily values <2mm
                regions_below_two = mask_matrix(product_array, 2, False, no_data_value)
                temp_mat = matrix_sum(temp_mat, regions_below_two)
                if isinstance(max_sequence, int):
                    max_sequence = temp_mat
                max_sequence[regions_below_two == 0] = np.maximum(max_sequence[regions_below_two == 0], temp_mat[regions_below_two == 0])
                temp_mat[regions_below_two == 0] = 0
            dataset = None

            if os.path.exists(cropped_product_path):
                os.remove(cropped_product_path)
        except AttributeError as attr_err:
            message = 'ERROR reading image. Aborting aggregation. Details: ' + str(attr_err)
            print(message)
            ciop.log('ERROR', message)
            sum_result = None
            count_above_one = None
            max_sequence = None
            mask_no_data_value = None
            projection = None
            geo_transform = None
            break
        if N_value == 30 or N_value == 60 or N_value == 90:
            max_sequence[temp_mat != 0] = np.maximum(max_sequence[temp_mat != 0], temp_mat[temp_mat != 0])
    return sum_result, count_above_one, max_sequence, mask_no_data_value, projection, geo_transform

In [24]:
def write_outputs(roi_name, first_date, last_date, sum_result, count_above_one, max_sequence, mask_no_data_value, image_format, product_count, projection, geo_transform, no_data_value):
    image_number = (datetime.strptime(last_date, '%Y-%m-%d') - datetime.strptime(first_date, '%Y-%m-%d')).days
    filenames = []
    filenames.append('CHIRPSv2_' + roi_name + '_N' + str(product_count) + '_daystotal_' + first_date + '_' + last_date + '.tif')
    write_output_image(filenames[0], sum_result, image_format, image_number, projection, geo_transform, mask_no_data_value, no_data_value)
    if product_count == 30 or product_count == 60 or product_count == 90:
        filenames.append('CHIRPSv2_' + roi_name + '_N' + str(product_count) + '_countaboveone_' + first_date + '_' + last_date + '.tif')
        filenames.append('CHIRPSv2_' + roi_name + '_N' + str(product_count) + '_dryspell_' + first_date + '_' + last_date + '.tif')
        write_output_image(filenames[1], count_above_one, image_format, image_number, projection, geo_transform, mask_no_data_value, no_data_value)
        write_output_image(filenames[2], max_sequence, image_format, image_number, projection, geo_transform, mask_no_data_value, no_data_value)
    return filenames

In [25]:
def write_anomaly_output(anomaly, first_date, last_date, lta_start_year, lta_end_year, aggregation, mask_no_value, N_value, regionOfInterest, roi_name, projection, geo_transform, no_data_value):
    image_number = (datetime.strptime(last_date, '%Y-%m-%d') - datetime.strptime(first_date, '%Y-%m-%d')).days
    filename = 'CHIRPSv2Anomaly_' + roi_name + '_N' + str(N_value) + '_' + aggregation + '_' + first_date + '_' + last_date + '_LTA' + str(lta_start_year) + '_' + str(lta_end_year) + '.tif'
    write_output_image(filename, anomaly, 'GTiff', image_number, projection, geo_transform, mask_no_value, no_data_value)
    return filename

In [26]:
def get_formatted_date(product_reference):
    metadata = ciop.search(end_point=product_reference,
                           params=[],
                           output_fields='identifier,startdate',
                           model="GeoTime")
    return metadata[0]['startdate']

In [27]:
def get_product(url, dest):

    r = requests.get(url)
    
    open(dest, 'wb').write(r.content)
    
    return r.status_code

In [28]:
def write_properties_file(dataframe, output_name):
    
    title = 'Output %s' % output_name
    first_date = get_formatted_date(dataframe.iloc[0]['self'])
    last_date = get_formatted_date(dataframe.iloc[-1]['self'])
    with open(output_name + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (first_date, last_date))
        file.write('geometry=%s' % (regionOfInterest['value']))

In [29]:
#Create folder to download the LTAs for the anomalies calculation
if not os.path.isdir('tmp_data'):
        os.mkdir('tmp_data')

In [30]:
nlist = [N_10['value'], N_30['value'], N_60['value'], N_90['value'], N_120['value'], N_150['value'], N_180['value'], N_270['value'], N_365['value']]
nvalue = [10, 30, 60, 90, 120, 150, 180, 270, 365]
nlist = [n=='True' for n in nlist]
product_years = gpd_final['startdate'].dt.year.unique()
for _year in product_years:
    products_data = gpd_final[(gpd_final['startdate'].dt.year == _year)]
    L = len(products_data.index.values)
    months_of_products = products_data['startdate'].dt.month.unique()
    for n in [index for index, value in enumerate(nlist) if value==True]:
        N = nvalue[n]
        n_months = N/30
        if n_months > len(months_of_products):
            print('Not enough products available for a aggregation of ' + str(N))
            break
        if N == 10:
            for month_idx in months_of_products:
                first_day = products_data[(products_data['startdate'].dt.month == month_idx)].iloc[0]['startdate'].day
                last_day = products_data[(products_data['startdate'].dt.month == month_idx)].iloc[-1]['startdate'].day
                for i in range(first_day, last_day, 10):
                    start_date = products_data[((products_data['startdate'].dt.month == month_idx) & (products_data['startdate'].dt.day == i))]['startdate'].tolist()[0]
                    if (i+9) < last_day:
                        end_date = products_data[((products_data['startdate'].dt.month == month_idx) & (products_data['startdate'].dt.day == i+9))]['startdate'].tolist()[0]
                    else:
                        end_date = products_data[((products_data['startdate'].dt.month == month_idx) & (products_data['startdate'].dt.day == last_day))]['startdate'].tolist()[0]
                    
                    interval_gpd = products_data[(products_data['startdate'] >= start_date) & (products_data['startdate'] <= end_date)]
                    interval_products = interval_gpd['enclosure'].tolist()
                    
                    start_day = start_date.day
                    start_month = start_date.month
                    end_day = end_date.day
                    end_month = end_date.month
                    
                    start_date = start_date.strftime('%Y-%m-%d')
                    end_date = end_date.strftime('%Y-%m-%d')
                    print(start_date)
                    print(end_date)
                    daily_sum, count_above_one, longest_sequence, no_value, projection, geo_transform = calc_aggregations(interval_products, N, region_of_interest)
                    
                    anomaly_files = []
                    #Query the data frame for the required LTAs
                    if daily_sum is not None:
                        LTA_daily_sum = get_lta_from_dataframe(lta_data, nameOfRegion, 'CHIRPSv2', 'daystotal', start_day, end_day, start_month, end_month, region_of_interest)
                        print(LTA_daily_sum)
                        if len(LTA_daily_sum) > 0:
                            LTA_daily_sum = LTA_daily_sum[0]
                            start_year = re.findall('\d{4}', os.path.basename(LTA_daily_sum))[0]
                            end_year = re.findall('\d{4}', os.path.basename(LTA_daily_sum))[1]
                            filepath = 'tmp_data/' + os.path.basename(LTA_daily_sum)
                            status = get_product(LTA_daily_sum, filepath)
                            if status == 200:
                                filepath = [filepath]
                                LTA_daily_sum = get_matrix_list(filepath)[0]
                                anomaly_daily_sum = np.divide(daily_sum, LTA_daily_sum)
                                filename = write_anomaly_output(anomaly_daily_sum, start_date, end_date, start_year, end_year, 'daystotal', no_value, N, region_of_interest, nameOfRegion, projection, geo_transform, -999.0)
                                anomaly_files.append(filename)
                                os.remove(filepath[0])
                        else:
                            period_str = str(start_month) + '-' + str(start_day) + '/' + str(end_month) + '-' + str(end_day)
                            message = 'No Long-term Average match found for "daystotal" aggregation from the period (month-day): ' + period_str 
                            ciop.log('INFO', message)
                    
                    if daily_sum is not None:
                        filenames = write_outputs(nameOfRegion, start_date, end_date, daily_sum, count_above_one, longest_sequence, no_value, 'GTiff', N, projection, geo_transform, -999.0)
                        for output_name in filenames:
                            write_properties_file(interval_gpd, output_name)

        else:
            if N == 30:
                for month_idx in months_of_products:
                    start_date = products_data[(products_data['startdate'].dt.month == month_idx)].iloc[0]['startdate']
                    end_date = products_data[(products_data['startdate'].dt.month == month_idx)].iloc[-1]['startdate']
                    
                    interval_gpd = products_data[(products_data['startdate'] >= start_date) & (products_data['startdate'] <= end_date)]
                    interval_products = interval_gpd['enclosure'].tolist()
                    
                    start_day = start_date.day
                    start_month = start_date.month
                    end_day = end_date.day
                    end_month = end_date.month
                    
                    start_date = start_date.strftime('%Y-%m-%d')
                    end_date = end_date.strftime('%Y-%m-%d')
                    print(start_date)
                    print(end_date)
                    daily_sum, count_above_one, longest_sequence, no_value, projection, geo_transform = calc_aggregations(interval_products, N, region_of_interest)
                    
                    
                    anomaly_files = []
                    
                    if daily_sum is not None:
                        LTA_daily_sum = get_lta_from_dataframe(lta_data, nameOfRegion, 'CHIRPSv2', 'daystotal', start_day, end_day, start_month, end_month, region_of_interest)
                        print(LTA_daily_sum)
                        if len(LTA_daily_sum) > 0:
                            LTA_daily_sum = LTA_daily_sum[0]
                            start_year = re.findall('\d{4}', os.path.basename(LTA_daily_sum))[0]
                            end_year = re.findall('\d{4}', os.path.basename(LTA_daily_sum))[1]
                            filepath = 'tmp_data/' + os.path.basename(LTA_daily_sum)
                            status = get_product(LTA_daily_sum, filepath)
                            if status == 200:
                                filepath = [filepath]
                                LTA_daily_sum = get_matrix_list(filepath)[0]
                                anomaly_daily_sum = np.divide(daily_sum, LTA_daily_sum)
                                filename = write_anomaly_output(anomaly_daily_sum, start_date, end_date, start_year, end_year, 'daystotal', no_value, N, region_of_interest, nameOfRegion, projection, geo_transform, -999.0)
                                anomaly_files.append(filename)
                                os.remove(filepath[0])
                        else:
                            period_str = str(start_month) + '-' + str(start_day) + '/' + str(end_month) + '-' + str(end_day)
                            message = 'No Long-term Average match found for "daystotal" aggregation from the period (month-day): ' + period_str 
                            ciop.log('INFO', message)
                        
                    if count_above_one is not None:
                        LTA_count_above_one = get_lta_from_dataframe(lta_data, nameOfRegion, 'CHIRPSv2', 'countaboveone', start_day, end_day, start_month, end_month, region_of_interest)
                        if len(LTA_count_above_one):
                            LTA_count_above_one = LTA_count_above_one[0]
                            start_year = re.findall('\d{4}', os.path.basename(LTA_count_above_one))[0]
                            end_year = re.findall('\d{4}', os.path.basename(LTA_count_above_one))[1]
                            filepath = 'tmp_data/' + os.path.basename(LTA_count_above_one)
                            status = get_product(LTA_count_above_one, filepath)
                            if status == 200:
                                filepath = [filepath]
                                LTA_count_above_one = get_matrix_list(filepath)[0]
                                anomaly_count_above_one = np.divide(count_above_one, LTA_count_above_one)
                                filename = write_anomaly_output(anomaly_count_above_one, start_date, end_date, start_year, end_year, 'countaboveone', no_value, N, region_of_interest, nameOfRegion, projection, geo_transform, -999.0)
                                anomaly_files.append(filename)
                                os.remove(filepath[0])
                        
                        else:
                            period_str = str(start_month) + '-' + str(start_day) + '/' + str(end_month) + '-' + str(end_day)
                            message = 'No Long-term Average match found for "countaboveone" aggregation from the period (month-day): ' + period_str 
                            ciop.log('INFO', message)
                    
                    if daily_sum is not None:
                        filenames = write_outputs(nameOfRegion, start_date, end_date, daily_sum, count_above_one, longest_sequence, no_value, 'GTiff', N, projection, geo_transform, -999.0)
                        filenames = filenames + anomaly_files
                        for output_name in filenames:
                            write_properties_file(interval_gpd, output_name)
                    
            else:
                for month_idx in range(months_of_products[0], months_of_products[-1], n_months):
                    start_date = products_data[(products_data['startdate'].dt.month == month_idx)].iloc[0]['startdate']
                    end_date = products_data[(products_data['startdate'].dt.month == month_idx + n_months-1)].iloc[-1]['startdate']
                    
                    interval_gpd = products_data[(products_data['startdate'] >= start_date) & (products_data['startdate'] <= end_date)]
                    interval_products = interval_gpd['enclosure'].tolist()
                    
                    start_day = start_date.day
                    start_month = start_date.month
                    end_day = end_date.day
                    end_month = end_date.month
                    
                    start_date = start_date.strftime('%Y-%m-%d')
                    end_date = end_date.strftime('%Y-%m-%d')
                    print(start_date)
                    print(end_date)
                    daily_sum, count_above_one, longest_sequence, no_value, projection, geo_transform = calc_aggregations(interval_products, N, region_of_interest)
                    
                    anomaly_files = []
                    
                    if daily_sum is not None:
                        LTA_daily_sum = get_lta_from_dataframe(lta_data, nameOfRegion, 'CHIRPSv2', 'daystotal', start_day, end_day, start_month, end_month, region_of_interest)
                        print(LTA_daily_sum)
                        if len(LTA_daily_sum) > 0:
                            LTA_daily_sum = LTA_daily_sum[0]
                            start_year = re.findall('\d{4}', os.path.basename(LTA_daily_sum))[0]
                            end_year = re.findall('\d{4}', os.path.basename(LTA_daily_sum))[1]
                            filepath = 'tmp_data/' + os.path.basename(LTA_daily_sum)
                            status = get_product(LTA_daily_sum, filepath)
                            if status == 200:
                                filepath = [filepath]
                                LTA_daily_sum = get_matrix_list(filepath)[0]
                                anomaly_daily_sum = np.divide(daily_sum, LTA_daily_sum)
                                filename = write_anomaly_output(anomaly_daily_sum, start_date, end_date, start_year, end_year, 'daystotal', no_value, N, region_of_interest, nameOfRegion, projection, geo_transform, -999.0)
                                anomaly_files.append(filename)
                                os.remove(filepath[0])
                        else:
                            period_str = str(start_month) + '-' + str(start_day) + '/' + str(end_month) + '-' + str(end_day)
                            message = 'No Long-term Average match found for "daystotal" aggregation from the period (month-day): ' + period_str 
                            ciop.log('INFO', message)
                        
                    if count_above_one is not None:
                        LTA_count_above_one = get_lta_from_dataframe(lta_data, nameOfRegion, 'CHIRPSv2', 'countaboveone', start_day, end_day, start_month, end_month, region_of_interest)[0]
                        if len(LTA_count_above_one):
                            LTA_count_above_one = LTA_count_above_one[0]
                            start_year = re.findall('\d{4}', os.path.basename(LTA_count_above_one))[0]
                            end_year = re.findall('\d{4}', os.path.basename(LTA_count_above_one))[1]
                            filepath = 'tmp_data/' + os.path.basename(LTA_count_above_one)
                            status = get_product(LTA_count_above_one, filepath)
                            if status == 200:
                                filepath = [filepath]
                                LTA_count_above_one = get_matrix_list(filepath)[0]
                                anomaly_count_above_one = np.divide(count_above_one, LTA_count_above_one)
                                filename = write_anomaly_output(anomaly_count_above_one, start_date, end_date, start_year, end_year, 'countaboveone', no_value, N, region_of_interest, nameOfRegion, projection, geo_transform, -999.0)
                                anomaly_files.append(filename)
                                os.remove(filepath[0])
                        
                        else:
                            period_str = str(start_month) + '-' + str(start_day) + '/' + str(end_month) + '-' + str(end_day)
                            message = 'No Long-term Average match found for "countaboveone" aggregation from the period (month-day): ' + period_str 
                            ciop.log('INFO', message)
                    
                    if daily_sum is not None:
                        filenames = write_outputs(nameOfRegion, start_date, end_date, daily_sum, count_above_one, longest_sequence, no_value, 'GTiff', N, projection, geo_transform, -999.0)
                        for output_name in filenames:
                            write_properties_file(interval_gpd, output_name)

2015-01-01
2015-01-10
chirps-v2.0.2015.01.01.tif
chirps-v2.0.2015.01.02.tif
chirps-v2.0.2015.01.03.tif
chirps-v2.0.2015.01.04.tif
chirps-v2.0.2015.01.05.tif
chirps-v2.0.2015.01.06.tif
chirps-v2.0.2015.01.07.tif
chirps-v2.0.2015.01.08.tif
chirps-v2.0.2015.01.09.tif
chirps-v2.0.2015.01.10.tif
[]


reporter:status:2019-03-29T17:31:04.351095 [INFO   ] [user process] No Long-term Average match found for "daystotal" aggregation from the period (month-day): 1-1/1-10
2019-03-29T17:31:04.351095 [INFO   ] [user process] No Long-term Average match found for "daystotal" aggregation from the period (month-day): 1-1/1-10


2015-01-11
2015-01-20
chirps-v2.0.2015.01.11.tif
chirps-v2.0.2015.01.12.tif
chirps-v2.0.2015.01.13.tif
chirps-v2.0.2015.01.14.tif
chirps-v2.0.2015.01.15.tif
chirps-v2.0.2015.01.16.tif
chirps-v2.0.2015.01.17.tif
chirps-v2.0.2015.01.18.tif
chirps-v2.0.2015.01.19.tif
chirps-v2.0.2015.01.20.tif
[]


reporter:status:2019-03-29T17:31:30.381068 [INFO   ] [user process] No Long-term Average match found for "daystotal" aggregation from the period (month-day): 1-11/1-20
2019-03-29T17:31:30.381068 [INFO   ] [user process] No Long-term Average match found for "daystotal" aggregation from the period (month-day): 1-11/1-20


2015-01-21
2015-01-30
chirps-v2.0.2015.01.21.tif
chirps-v2.0.2015.01.22.tif
chirps-v2.0.2015.01.23.tif
chirps-v2.0.2015.01.24.tif
chirps-v2.0.2015.01.25.tif
chirps-v2.0.2015.01.26.tif
chirps-v2.0.2015.01.27.tif
chirps-v2.0.2015.01.28.tif
chirps-v2.0.2015.01.29.tif
chirps-v2.0.2015.01.30.tif
[]


reporter:status:2019-03-29T17:31:56.040932 [INFO   ] [user process] No Long-term Average match found for "daystotal" aggregation from the period (month-day): 1-21/1-30
2019-03-29T17:31:56.040932 [INFO   ] [user process] No Long-term Average match found for "daystotal" aggregation from the period (month-day): 1-21/1-30


2015-01-01
2015-01-31
chirps-v2.0.2015.01.01.tif
chirps-v2.0.2015.01.02.tif
chirps-v2.0.2015.01.03.tif
chirps-v2.0.2015.01.04.tif
chirps-v2.0.2015.01.05.tif
chirps-v2.0.2015.01.06.tif
chirps-v2.0.2015.01.07.tif
chirps-v2.0.2015.01.08.tif
chirps-v2.0.2015.01.09.tif
chirps-v2.0.2015.01.10.tif
chirps-v2.0.2015.01.11.tif
chirps-v2.0.2015.01.12.tif
chirps-v2.0.2015.01.13.tif
chirps-v2.0.2015.01.14.tif
chirps-v2.0.2015.01.15.tif
chirps-v2.0.2015.01.16.tif
chirps-v2.0.2015.01.17.tif
chirps-v2.0.2015.01.18.tif
chirps-v2.0.2015.01.19.tif
chirps-v2.0.2015.01.20.tif
chirps-v2.0.2015.01.21.tif
chirps-v2.0.2015.01.22.tif
chirps-v2.0.2015.01.23.tif
chirps-v2.0.2015.01.24.tif
chirps-v2.0.2015.01.25.tif
chirps-v2.0.2015.01.26.tif
chirps-v2.0.2015.01.27.tif
chirps-v2.0.2015.01.28.tif
chirps-v2.0.2015.01.29.tif
chirps-v2.0.2015.01.30.tif
chirps-v2.0.2015.01.31.tif
['https://store.terradue.com/better-wfp-00009/_results/workflows/ec_better_wfp_01_03_04_wfp_01_03_04_1_5/run/3f0b6d68-4410-11e9-8661-0242ac11

/opt/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:103: RuntimeWarning: invalid value encountered in divide
/opt/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:123: RuntimeWarning: invalid value encountered in divide


In [31]:
'''
from matplotlib import pyplot
%matplotlib inline


fig0 = pyplot.figure()
ax0 = fig0.add_subplot(111)
cax0 = ax0.matshow(daily_sum)
fig0.colorbar(cax0)

fig1 = pyplot.figure()
ax1 = fig1.add_subplot(111)
cax1 = ax1.matshow(count_above_one)
fig1.colorbar(cax1)


fig = pyplot.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(no_value)
fig.colorbar(cax)


fig2 = pyplot.figure()
ax2 = fig2.add_subplot(111)
cax2 = ax2.matshow(longest_sequence)
fig2.colorbar(cax2)

pyplot.show()
'''

'\nfrom matplotlib import pyplot\n%matplotlib inline\n\n\nfig0 = pyplot.figure()\nax0 = fig0.add_subplot(111)\ncax0 = ax0.matshow(daily_sum)\nfig0.colorbar(cax0)\n\nfig1 = pyplot.figure()\nax1 = fig1.add_subplot(111)\ncax1 = ax1.matshow(count_above_one)\nfig1.colorbar(cax1)\n\n\nfig = pyplot.figure()\nax = fig.add_subplot(111)\ncax = ax.matshow(no_value)\nfig.colorbar(cax)\n\n\nfig2 = pyplot.figure()\nax2 = fig2.add_subplot(111)\ncax2 = ax2.matshow(longest_sequence)\nfig2.colorbar(cax2)\n\npyplot.show()\n'